In [1]:
# Cell 1 – imports & helper
import pandas as pd, geopandas as gpd
import folium
from folium.plugins import TimestampedGeoJson
from geopy.distance import geodesic
from pathlib import Path

DATA   = Path("../data")
FIGS   = Path("../figs"); FIGS.mkdir(exist_ok=True)

def gc_km(a, b):
    return geodesic(a, b).km

In [2]:
# Cell 2 – load detections & pick a tag
det = pd.read_csv(DATA / "blue_shark_detections.csv",
                  parse_dates=["datecollected"])          # timestamp column name
det.rename(columns={"datecollected": "timestamp"}, inplace=True)

tag_id = det.tagname.value_counts().idxmax()
fish   = det[det.tagname == tag_id].copy().sort_values("timestamp")
print(f"Working with tag {tag_id}  –  {len(fish)} detections")

Working with tag A69-9001-24395  –  526 detections


In [3]:
# Cell 3 – residency % on the Halifax Line
line_mask     = fish["station"].str.contains("HFX", case=False)
residency_pct = round(line_mask.mean() * 100, 1)
print(f"Residency on Halifax Line = {residency_pct} %")

Residency on Halifax Line = 100.0 %


In [4]:
# Cell 4 – build a time-slider map
gdf = gpd.GeoDataFrame(
    fish,
    geometry=gpd.points_from_xy(fish.longitude, fish.latitude),
    crs="EPSG:4326"
)

m = folium.Map(
    location=[gdf.geometry.y.mean(), gdf.geometry.x.mean()],
    zoom_start=6
)

features = [
    {
        "type": "Feature",
        "geometry": {"type": "Point",
                     "coordinates": [row.longitude, row.latitude]},
        "properties": {
            "time": row.timestamp.isoformat(),
            "popup": f"{row.station}<br>{row.timestamp}"
        },
    }
    for row in gdf.itertuples()
]

TimestampedGeoJson(
    {"type": "FeatureCollection", "features": features},
    period="PT1H", add_last_point=True
).add_to(m)

out_html = FIGS / f"track_map_{tag_id}.html"
m.save(out_html)
out_html    # shows clickable link in Jupyter

PosixPath('../figs/track_map_A69-9001-24395.html')

In [5]:
m